<h1>Tuto motion planing RRT avec pinocchio </h1>

Dans ce tutoriel, nous allons voir comment faire un algorithme rrt avec pinocchio

Tout d'abord nous aurons besoin de charger le modèle du robot qui sera le bras robotique ur5, placer des obstacles dans notre environnement, placer une cible , faire une fonction qui nous permet de mesurer la distance entre l'organe terminale et la cible, une fonction qui nous permet de vérifier les collisions pour une configuration donnée, une qui nous permet de produire une configuration aléatoire sans collision. Aussi une fonction qui nous permet de vérifier les collisions lors du passage d'une configuration à une autre et enfin notre fonction qui exécutera l'algorithme rrt. 

In [7]:
import pinocchio as pin
import numpy as np
from numpy.linalg import norm
import graphviz
from example_robot_data import load 
import itertools
import time

In [8]:
robot = load('ur5')  # Chargement du bras robotique

Création des obstacles en capsule: 

In [9]:
color = [ 1.0, 0.2, 0.2, 1.0 ]                       # couleur de la capsule rgbt
rad,length = .08,0.4                                  # rayon et longueur de la capsule
R = np.array([[1.,0.,0.],[0.,0.,-1.],[0.,1.,0.]])  # Matrice de rotation (orientation de capsule)

Positions = [[0.6, 0., 0.2 ],[0.3,0.1,0.6],[0.8,-0.1,0.1],[0.3,0.4,0.4]]      # Vecteurs de translation des obstacles 
for i,P in enumerate(Positions):
    obstacle = pin.GeometryObject.CreateCapsule(rad,length)  # Pinocchio obstacle object
    obstacle.meshColor = np.array(color)    # Couleur de l'obstacle 
    obstacle.name = "obstacle%d"%i                             # nom de l'objet
    obstacle.parentJoint = 0                                 # objet parent = 0 = universe
    obstacle.placement = pin.SE3(R,np.array(P))  # fonction permettant de d'avoir un élément de SE3
    robot.collision_model.addGeometryObject(obstacle)  # Ajout de l'objet dans le modèle de  collision
    robot.visual_model.addGeometryObject(obstacle)  # Ajout de l'objet dans le modèle de visuel

Création des paires de collision :

In [10]:
nObstacle = len(Positions)                                  # Nombre d'obstacle
nCorps = robot.collision_model.ngeoms - nObstacle           # Nombre de corps du robot
indiceCorps = range(nCorps)                                 # Indice des corps du robot
indiceCorpsEnv = range(nCorps,nCorps+nObstacle)             # Indice des corps de l'environnement du robot
robot.collision_model.removeAllCollisionPairs()             # Suppression des anciens couple de collision
for j,k in itertools.product(indiceCorps,indiceCorpsEnv):
    robot.collision_model.addCollisionPair(pin.CollisionPair(j,k)) # Ajout des paires de collision

robot.collision_data = pin.GeometryData(robot.collision_model)         # Génération des données de collision
robot.visual_data = pin.GeometryData(robot.visual_model)               # Génération des données visuelles


In [11]:
robot.initViewer(loadModel = True)
robot.display(robot.q0)

Ajoutons notre cible :

In [33]:
rgbt = [0.2, 1.2, 0.2, 1.0]  # rouge, vert, bleu, transparence (la couleur de l'objet)
robot.viewer.gui.addSphere("world/target", .05, rgbt)  # .05 représente le rayon de la sphère 
target= [0.5,0.2,0.4]       # Coordonnée de la sphère
q1 = (target[0], target[1],target[2], 1, 0, 0, 0)  # x, y, z, quaternion
robot.viewer.gui.applyConfiguration("world/target", q1) # Création de la sphère
robot.viewer.gui.refresh()  # Pour rafraîchir la fenêtre.

In [13]:
def endEffPos(q):                                               # Nous avons besoin de savoir où se trouve l'organe terminal pour chaque configuration
    pin.framesForwardKinematics(robot.model, robot.data, q)     # qui sera appliquée sur le bras robotique
    return robot.data.oMf[-1].translation                       # Pour cela on calcule la cinématique direct et on en déduit l'emplacement du repère de l'organe terminal
                                                                # La partie translation correspond à sa position

In [14]:
# Ensuite nous aurons aussi besoin de connaître la distance qui sépare la cible de l'organe terminal
def dist(q,target):
    return norm(endEffPos(q) - target)                     # Pour cela faut juste calculer la norme de la difference de leur vecteur position


In [15]:
def collision(q):                                                    # Comme l'environnement du bras robotique contient des obstacles on aura besoin de verifier si une des parties du bras
                                                                # rentre en collision avec un obstacle et agir en conséquence                    
    pin.updateGeometryPlacements(robot.model, robot.data, robot.collision_model, robot.collision_data, q)
    return pin.computeCollisions(robot.collision_model, robot.collision_data, False)

In [16]:
def randomQ(check=False):                                         # Pour l'algorithme de recherche on auras besoin de générer des configurations aléatoire sans collision
    while True:
        q = np.random.rand(robot.model.nq) * 6.4 - 3.2
        if not check or not collision(q): return q

In [17]:
def lilqrand(range,target):                                               # Permet de produire une configuration aléatoire sans collision au voisinage("range") de la cible   
    while True:
        q = randomQ(check=True)
        if dist(q,target) < range:
            return q

In [18]:
def extendtoconf(q1, q2):                                       # Vérifie s'il y'a un collision entre q1 et q2 et 
                                                                # retourne la configuration juste avant la collision
    dq = (q2 - q1) * .05
    for i in range(20):
        if not collision(q1 + dq):
            q1 = q1 + dq
        else:
            return q1
    return q1

Nous allons essayer de modéliser les résultats produits en graph et pour cela nous allons créer une structure de noeud dans laquelle les différentes configurations seront stockés. Et pour la création du graph nous allons utiliser la bibliothéque graphviz.

In [19]:
class Node:

    def __init__(self, data):
        self.data = data
        self.children = []
        self.parent = None

    def set_parent(noderef, nodeadd):
        noderef.parent = nodeadd

    def set_children(noderef, nodeadd):
        noderef.children.append(nodeadd)

Maintenant que nous avons presque toutes les fonctions dont nous aurons besoin, nous pouvons nous lancer dans la conception du rrt.
L'idée serait de faire en sorte que le bras robotique trouve une suite de configuration sans collision qui lui permettrons d'attraper la cible par son organe terminal partant d'une configuration initiale à définir.
Donc nous allons produire à chaque fois une configuration aléatoire, chercher la configuration la plus proche de celle là parmi celle déjà produite. Tester le trajet entre les deux configurations et s'il y'a une collision durant le trajet la configuration juste avant collision est retenue et ajouter dans l'arbre.
Et une fois sur trois on biaise l'aléatoire en faisant en sorte que la configuration soit une dont l'organe terminal se trouve aux environs de la cible. Et le processus est répété jusqu'à ce que la cible soit trouvé. Théoriquement s'il existe une solution l'algorithme la trouvera mais dans un temps qui peut tendre vers l'infini. Donc pour certains cas l'algorithme risque de vraiment s'executer pendant longtemps.

Pour avoir une représentation graphique du développement de l'arbre nous utiliserons graphviz

In [20]:
def minidx(tab):                                                # Retourne l'indice du minimum d'un tableau
    for i in range(len(tab)):
        if tab[i] == min(tab):
            return i

In [21]:
def rrt(
        g,target,q0=None):                                 # g est l'instance de graphviz qui nous permettra de créer notre graphe
    rhist = []
    
    q = randomQ(check=True) if q0 is None else q0
    hist = [Node(q.copy())]                         # les configurations sont enregistrées dans des nœuds
    baseconf = str(sum(q))
    g.node(baseconf, 'BaseConf',color = 'blue')                    # Ajout de la configuration de départ dans le graphe
    t = 1
    while True:
        if t % 3 == 0:
            qr = lilqrand(0.05,target)                             # Génère une configuration random dans laquelle l'OT est à 0.05 de la cible 
                                                                   # Vous pouvez vous amusez de changer la valeur 0.05 et voire le nombre de configuration qui l'amène jusqu'à la cible
                                                                   # Ainsi que le temps d'execution de l'algorithme, tout ça en prenant en compte "la complexité" de l'emplacement de la cible par rapport à l'obstacle
        else:
            qr = randomQ(check=True)
        t = t + 1
        myarray = []
        norms = []
        for i in range(len(hist)):
            norms.append(norm(hist[i].data - qr))
        idx = minidx(norms)
        qt = extendtoconf(hist[idx].data, qr)                 # Vérifie s'il y'a une collision entre la nouvelle config produite et celle qui est la plus proche d'elle et renvoie la dernière config tout juste avant la collision
        if sum(qt) != sum(qr):                                # Pour éviter de remettre la même config dans le graphe
            node = Node(qt.copy())
            Node.set_children(hist[idx], node)
            Node.set_parent(node, hist[idx])
            hist.append(node)                                 # Mise a jour de l'arbre
            g.node(str(sum(node.data)), str(len(hist) - 1))
            if node.parent is not None and sum(node.parent.data) != sum(node.data):
                g.edge(str(sum(node.parent.data)), str(sum(node.data)))
            if dist(node.data,target) < 5e-2:                                        # Si la distance entre la cible et l'organe terminal est inférieur à 0.05 on considère que l'organe terminal à attraper la cible       
                g.node(str(sum(hist[-1].data)), str(len(hist)), color='green')
                tmp = node
                while tmp is not None:
                    rhist.append(tmp)
                    tmp = tmp.parent
                for i in range(len(rhist)):
                    if rhist[i].parent is not None:
                        g.edge(str(sum(rhist[i].parent.data)), str(sum(rhist[i].data)), color='green')
                print('Trajectoire touvee !')
                return rhist
            
    return None 

Passons maintenant au test de notre algorithme. Pour cela on va créer notre graphe et choisir une configuration sans collision de départ

In [34]:
g = graphviz.Digraph('rrt', filename= 'Test5rrt.gv')

In [23]:
qInit = randomQ(check= True)

In [35]:
robot.display(qInit)

In [36]:
chemin = rrt(g,target,qInit)

Trajectoire touvee !


In [37]:
len(chemin)

6

Maintenant après avoir trouver la suite de configuration qui nous amène jusqu'à la cible nous pouvons visualiser le chemin dans gepetto-gui. Et pour cela on va faire une petite fonction : 

In [30]:
def disptraj(rhist):                                          # Cette fonction permet de montrer la trajectoire du bras dans le veiwer
    for i in range(1, len(rhist) + 1):
        q = rhist[-i].data
        if -i - 1 >= -len(rhist):
            dq = (rhist[-i - 1].data - rhist[-i].data) * .05
            for j in range(20):
                q = q + dq
                robot.display(q)
                time.sleep(5e-2)                       

In [38]:
disptraj(chemin)

In [39]:
g.engine = 'neato'     # Paramètre de la forme de l'arbre
g.view()               # Permet de visualiser l'arbre produit dans un fichier pdf qui va s'ouvrir automatiquement

'Test5rrt.gv.pdf'


(evince:11928): Gtk-WARNING **: 10:08:52.500: Could not load a pixbuf from icon theme.
This may indicate that pixbuf loaders or the mime database could not be found.


Pour conclure nous pourrons juste dire que le temps d'execution de l'algorithme à trouver un résultat dépend de la manière dont on a biaiser l'aléatoire (le rayon du voisinage de la cible) mais aussi de la complexité de l'emplacement de la cible par rapport aux obstacles. Vous pourrez aussi essayer de changer la taille des obstacles pour voir ce que ça donne.    